In [1]:
from dino_utils import set_device, dino_processor, setup_dino_model, calculate_embeddings

Using cuda as device
Using cuda as device


/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis/dinov2/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
import pickle
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [3]:
import torch
import os
from glob import glob
from tqdm import tqdm
import numpy as np

In [4]:
# device = set_device()
# dino_model = setup_dino_model(device)

### Compute Embeddings for Real Images

In [5]:
# Real Images
input_images_path = f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/"
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/real_images_embeddings.pt"
calculate_embeddings(input_images_path, save_path)

Embeddings for /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/all/ already calculated


### Compute Embeddings for Hyperstyle Images

In [6]:
os.chdir(f'{ROOT_PATH}/2_Inversion/hyperstyle/')
from hyperstyle_utils import load_hyperstyle, load_generator_inputs, generate_hyperstyle
os.chdir(f"{current_wd}")

In [7]:
skus = glob(f"{DATA_PATH}/Zalando_Germany_Dataset/dresses/images/e4e_images/all/*.jpg")
skus = [elem.split('/')[-1].split('.')[0] for elem in skus]

In [8]:
# G = load_hyperstyle()
# calculate_embeddings(skus[:10], 't.pt', generator_type='hyperstyle', generator = G)

In [9]:
G = load_hyperstyle()
save_path = f"{DATA_PATH}/Models/Assessor/DinoV2/Embeddings/hyperstyle_embeddings.pt"
calculate_embeddings(skus, save_path, generator_type='hyperstyle', generator=G)

Loading HyperStyle from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/hyperstyle/00005_snapshot_1200_restyle_77000/resume/checkpoints/best_model.pt
Loading pretrained W encoder...
Loading ReStyle e4e from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/restyle/00005_snapshot_1200/checkpoints/best_model.pt
Model successfully loaded!


  0%|          | 0/14060 [00:00<?, ?it/s]/pfs/work7/workspace/scratch/tu_zxmav84-thesis/miniconda3/envs/thesis/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/pfs/work7/workspace/scratch/tu_zxmav84-thesis/miniconda3/envs/thesis/lib/python3.7/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/ups